# This section intends to describe the problem

## Due to the sortage of the data, this section still uses the data provided suggested by the program again.

### The following is the description of the problem.

we explored New York City and the city of Toronto and segmented and clustered their neighborhoods. Both cities are very diverse and are the financial capitals of their respective countries. One interesting idea would be to compare the neighborhoods of the two cities and determine how similar or dissimilar they are. Is New York City more like Toronto or Paris or some other multicultural city? I will leave it to you to refine this idea.

In a city of your choice, if someone is looking to open a restaurant, where would you recommend that they open it? Similarly, if a contractor is trying to start their own business, where would you recommend that they setup their office?


# Introduction

The major purpose of this project intends to help the client to start a new business in a metropolitan city such as Toronto. 

# Background

It is very important to find the best location for small business. However, most clients do not have the resource to find the best location. Most of them decide the location of the store based on their own experience. This pose great challenage for the location. Moreover, most clients only have limited budget, therefor, it is essential to analyze the suitable location before making the decision. 

# Business Problem

In this project, the author intends to help the client make the correct decision based on the information of the specific city, such as Toronto. The challenge is to find the best location and also make the correct decision so that the client make money as soon as possible. 

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [4]:
file_location = r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [5]:
data = requests.get(file_location).text

In [6]:
data_c91 = BeautifulSoup(data, 'lxml')

In [7]:
data_columns = ['Postalcode','Borough','Neighborhood']
toronto_data = pd.DataFrame(columns = data_columns)

In [8]:
data_content = data_c91.find('div', class_='mw-parser-output')
formal_data = data_content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

In [9]:
for tr in formal_data.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto_data = toronto_data.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [10]:
toronto_data = toronto_data[toronto_data.Borough!='Not assigned']
toronto_data = toronto_data[toronto_data.Borough!= 0]
toronto_data.reset_index(drop = True, inplace = True)
j = 0
for j in range(0,toronto_data.shape[0]):
    if toronto_data.iloc[j][2] == 'Not assigned':
        toronto_data.iloc[j][2] = toronto_data.iloc[j][1]
        j = j+1
                                 
df = toronto_data.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Clean data

In [11]:
df = df.dropna()
data_nan = 'Not assigned'
df = df[(df.Postalcode != data_nan ) & (df.Borough != data_nan) & (df.Neighborhood != data_nan)]

In [12]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [14]:
print(df2.shape)
df2.head()

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# coordinates

In [29]:
file_location2 = r'C:\Users\antun\Pictures\Numerical document\Python\Python_pandas\Training_1\Data_file\Geospatial_Coordinates.csv'

In [30]:
coord_data = pd.read_csv(file_location2)

In [34]:
df_final = pd.concat([df2, coord_data], axis=1)
df_final = df_final.drop(['Postal Code'], axis = 1)
df_final.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [36]:
df_final.to_csv(r'C:\Users\antun\Pictures\Numerical document\Python\Python_pandas\Training_1\Data_file\df_final.csv')

# The final project

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import folium
import locale
from geopy.geocoders import Nominatim

In [67]:
city = 'Toronto, Ontario'
location = geolocator.geocode(city)
location_latitude = location.latitude
location_longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are', latitude, ',' ,longitude)

The geograpical coordinate of Toronto, Canada are 43.653963 , -79.387207


In [68]:
data_toronto = df_final[df_final['Borough'].str.contains('Toronto')].reset_index(drop=True)
data_toronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [69]:
location_toronto = folium.Map(location=[location_latitude, location_longitude], zoom_start=13)

for lat, lng, borough, neighborhood in zip(data_toronto['Latitude'], data_toronto['Longitude'], data_toronto['Borough'], data_toronto['Neighborhood']):
  label = '{},{}'.format(neighborhood,borough)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
      [lat, lng],
      radius = 5,
      popup = label,
      color = 'black',
      fill = True,
      fill_color = '#3186cc',
      fill_opacity = 0.7).add_to(location_toronto) 

In [72]:
CLIENT_ID = '1HABFONPEEOMT0AS2GVXPK53A4EO43AXRTJVN3B03CG3N3BE'
CLIENT_SECRET = '0CR2ZL5CZHLX5OSIIJRZCBPXGCC1TIS2HQ04CUYLFBFRBG1Q'
VERSION = '20180605'
radius = 200
LIMIT = 50

In [75]:
def City_Venues(names, city_latitudes, city_longitudes, radius=200):
    venues_list=[]
    for name, lat, lng in zip(names, city_latitudes, city_longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return( nearby_venues)
toronto_venues = City_Venues(names=df_toronto['Neighborhood'],
                                   city_latitudes=df_toronto['Latitude'],
                                   city_longitudes=df_toronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [46]:
toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,15,15,15,15,15,15
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15


In [76]:
unique_values = len(toronto_venues['Venue Category'].unique())
print('There are {} unique categories.'.format(unique_values))

There are 132 unique categories.


In [80]:
city_mock = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
city_mock['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [city_mock.columns[-1]] + list(city_mock.columns[:-1])
city_mock = city_mock[fixed_columns]
city_mock.head()

,Yoga Studio,Adult Boutique,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bank,...,Taco Place,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
torontogroup = city_mock.groupby('Neighborhood').mean().reset_index()
torontogroup.head()

,Neighborhood,Yoga Studio,Adult Boutique,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,...,Taco Place,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.000000,...,0.0,0.041667,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.0
1,"Brockton, Exhibition Place, Parkdale Village",0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.000000,0.5,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,"Cabbagetown, St. James Town",0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.083333,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [87]:
def best_venues(row, num_top_venues):
    row_suitable = row.iloc[1:]
    row_suitable_sorted = row_suitable.sort_values(ascending=False)
    
    return row_suitable_sorted.index.values[0:num_top_venues]

In [91]:
num_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
near_venues_sorted = pd.DataFrame(columns=columns)
near_venues_sorted['Neighborhood'] = torontogroup['Neighborhood']

for ind in np.arange(torontogroup.shape[0]):
    near_venues_sorted.iloc[ind, 1:] = best_venues(torontogroup.iloc[ind, :], num_top_venues)

near_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Steakhouse,Coffee Shop,Asian Restaurant,General Travel,Pharmacy,Japanese Restaurant,Park,Hotel,Bar,Food Court
1,"Brockton, Exhibition Place, Parkdale Village",Gym / Fitness Center,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Costume Shop,Creperie,Cuban Restaurant,Deli / Bodega
2,Business Reply Mail Processing Centre 969 Eastern,Brewery,Auto Workshop,Wine Bar,Deli / Bodega,Ethiopian Restaurant,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Cuban Restaurant
3,"CN Tower, Bathurst Quay, Island airport, Harbo...",Performing Arts Venue,Farmers Market,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Costume Shop,Creperie,Cuban Restaurant
4,"Cabbagetown, St. James Town",Café,Beer Store,Market,General Entertainment,Coffee Shop,Restaurant,Pizza Place,Diner,Bakery,Italian Restaurant
5,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Pharmacy,Middle Eastern Restaurant,Café,Bookstore,Smoothie Shop,Fried Chicken Joint,French Restaurant
6,"Chinatown, Grange Park, Kensington Market",Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Caribbean Restaurant,Dumpling Restaurant,Mexican Restaurant,Bar,Wine Bar,Noodle House,Cheese Shop
7,Christie,Japanese Restaurant,Nightclub,Wine Bar,Cuban Restaurant,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Deli / Bodega,Creperie
8,Church and Wellesley,Burger Joint,Martial Arts Dojo,Salon / Barbershop,Bubble Tea Shop,Japanese Restaurant,Breakfast Spot,Boutique,Diner,Dessert Shop,Flower Shop
9,"Commerce Court, Victoria Hotel",Coffee Shop,Deli / Bodega,Restaurant,Café,Hotel,Bank,Sandwich Place,Salad Place,Gym,Breakfast Spot


In [95]:
kclusters = 7
toronto_clustering = torontogroup.drop('Neighborhood', 1)
from sklearn.cluster import KMeans
kNN_means = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)
kNN_means.labels_[0:7]

array([1, 6, 1, 4, 1, 1, 1])

In [98]:
toronto_merged = df_toronto
toronto_merged['Cluster Labels'] = kmeans.labels_
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Other Great Outdoors,Trail,Park,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Ethiopian Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Wine Bar,Farmers Market,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Costume Shop,Creperie,Cuban Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,4,Park,Fish & Chips Shop,Cuban Restaurant,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Deli / Bodega,Creperie,Farmers Market
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Café,Bakery,Seafood Restaurant,Cheese Shop,Bookstore,Ice Cream Shop,Bank,Bar,Clothing Store
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Lake,Wine Bar,Deli / Bodega,Ethiopian Restaurant,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Cuban Restaurant,Fast Food Restaurant


In [66]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

# Results and conclusion